In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from loguru import logger
import yaml
import sys
from pyspark.sql import SparkSession
import pandas as pd

from mlops_course.config import ProjectConfig
from mlops_course.data_processor import DataProcessor
from marvelous.logging import setup_logging
from marvelous.timer import Timer

config = ProjectConfig.from_yaml(config_path="../project_config.yml", env="dev")

setup_logging(log_file="logs/marvelous-1.log")

logger.info("Configuration loaded:")
logger.info(yaml.dump(config, default_flow_style=False))